In [ ]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.utils.hdf5_io import DataStorage
from mesostat.metric.metric import MetricCalculator
from mesostat.stat.anova import as_pandas_lst, anova_homebrew
from mesostat.visualization.mpl_matrix import imshow

from lib.gallegosalas.data_fc_db_sessional import DataFCDatabase
from lib.sych.plot_helper import imshow_dataset_by_mouse, imshow_dataset_by_session
from lib.sych.metric_helper import metric_by_session

%load_ext autoreload
%autoreload 2

In [ ]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata2/'

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
dataDB.read_neuro_files()

In [ ]:
dataDB.baseline_normalization()

In [ ]:
ds = DataStorage('gallerosalas_result_individual_region.h5')

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

# 2. Activity

In [ ]:
for metricName in ['mean', 'std', 'avg_entropy']:
    for iMouse, mousename in enumerate(sorted(dataDB.mice)):
        for datatype in dataDB.get_data_types():
            dataName = metricName + '_' + datatype
            print(dataName)
            metric_by_session(dataDB, mc, ds, mousename, metricName, 'p', datatype=datatype, dataName=dataName)

In [ ]:
for metricName in ['mean', 'std', 'avg_entropy']:
    for datatype in dataDB.get_data_types():
        dataName = metricName + '_' + datatype
        imshow_dataset_by_mouse(dataDB, ds, dataName, havePerf=False)

# 3. Temporal Structure
## 3.1 Temporal specificity

In [ ]:
import multiprocessing
from time import time
from scipy.stats import wilcoxon
from mesostat.utils.arrays import numpy_transpose_byorder

def task2(data2D):
    dataTmp = data2D - np.mean(data2D)
    
    nTime = data2D.shape[1]
    rez = np.zeros(nTime)

    for iTime in range(nTime):
        T, pval = wilcoxon(dataTmp[:, iTime])
#         rez[iTime] = np.sign(np.mean(dataTmp[:, iTime])) * int(pval < 0.01 / nTime)
        rez[iTime] = np.sign(np.mean(dataTmp[:, iTime])) * (-np.log10(pval))
    return rez
    

# Parallel stuff
pool = multiprocessing.Pool(processes=4)

for iMouse, mousename in enumerate(sorted(dataDB.mice)):
    for datatype in dataDB.get_data_types():
        print('doing', mousename, datatype)
        rows = dataDB.get_rows('neuro', {'mousename' : mousename, 'datatype' : datatype})

        fig, ax = plt.subplots(ncols=len(rows), figsize=(6*len(rows), 3), tight_layout=True)
        fig.suptitle(mousename)

        for iPlot, (idx, row) in enumerate(rows.iterrows()):
            session = row['session']
            print('..session', session, end='')
            timeStart = time()

            data = dataDB.dataNeuronal[idx]
            dataTmp = numpy_transpose_byorder(data, 'rsp', 'prs')
            rez = np.array(pool.map(task2, dataTmp))

    #         ax[iPlot].imshow(rez, vmin=-1.1, vmax=1.1, cmap='jet')
            ax[iPlot].imshow(rez, vmin=-10, vmax=10, cmap='jet')
            ax[iPlot].set_title(session)

            print('; took time', time() - timeStart)
        plt.show()
    
pool.terminate()
pool.join()

# 4. Behaviour-Specific performance

## 4.1 Temporal mean vs learning

In [ ]:
for iMouse, mousename in enumerate(sorted(dataDB.mice)):
    for datatype in dataDB.get_data_types():
        dataName = 'ord_mean' + '_' + datatype
        print(dataName)
        metric_by_session(dataDB, mc, ds, mousename, 'ord_mean', 'p', datatype=datatype, dataName=dataName)

In [ ]:
for datatype in dataDB.get_data_types():
    dataName = 'ord_mean' + '_' + datatype
    imshow_dataset_by_mouse(dataDB, ds, dataName, havePerf=False)